# 2. hipotesiko CSVak sortzeko eskuliburua

Koaderno honetan paper-eko datuak eskuratzeko erabili dugun prozedura azaltzen da eta gainera, azpiko gelaxkak exekutatuta berregin daitezke guk jarraitutako pausoak.

## Definitu jarraian ageri diren azpiprogramak

### Grafoen aurreprozesaketaren azpiprograma

In [ ]:
#Sarrera: G grafo orijinala, k sortu nahi den partizio kopurua
#Irteera: G grafo aurreprozesatua, kodeak hiztegia, pisuakGuztira grafo prozesatuko pisuen batura eta verbose exekuzioari buruzko informazioa duen zerrenda. 
def prozesatu_grafoa(G,k,weight = 'weight'):

    GO = G.copy() #Kopiatu grafo orijinala.

    kodeak = { } #Hasieratu string-zenbaki oso erreferetziak izango dituen hiztegia. 
    pisuakGuztira = 0 #Grafoko ertzen pisuen batura gordeko duen aldagaia hasieratu. 
    
    multzoak = [ ] #Grafoko multzo isolatuak gordeko dituen zerrenda hasieratu.
    aztertuGabeak = list(GO.nodes) #Nodo aztertu gabeen zerrenda hasieran nodo guztiek osatuko dute.
    ilara = [ ]
    verbose = [ ] #Exekuzioari buruzko informazioa biltzeko.
    ##1. FASEA: Grafoko multzo isolatuak topatu eta multzoak zerrendan gorde.
    while (len(aztertuGabeak) > 0): #Multzo guztiak identifikatzeko.
      unekoa = aztertuGabeak[0]  
      ilara.extend(GO.edges(unekoa))
      aztertuGabeak.pop(0)
      multzoa = [unekoa]
      while (len(ilara) > 0): #Zabalerako korritzea erabiltzen da. Multzo baten nodoak identifikatu.
        unekoa = list(ilara.pop(0))[1]
        if (unekoa in aztertuGabeak):
          ilara.extend(GO.edges(unekoa)) #Uneko nodoaren ertzak gehitu ilarara
          aztertuGabeak.remove(unekoa) #Aztertu gabeetatik kendu unekoa.
          multzoa.append(unekoa)
      multzoak.append(multzoa)
    
    #2. FASEA: Zein multzo baztertu erabakitzea.
    listaHandienaTam = len(max(multzoak, key = len)) #Topatu den multzo handienaren nodo kopurua. 
    verbose.append(listaHandienaTam)
    baztergarritasuna = listaHandienaTam/k #Multzo bat baztertua izango da atalase balio hau gainditzen ez badu, hau da, multzo handienaren tamaina zati partizio kopurua.
    verbose.append(baztergarritasuna)
    kendu = [ ]
    maxim=0 
    for i in multzoak: #Atalasea baino txikiagoak diren multzoak baztertzea. Baztertuko diren multzoen nodoak kendu zerrendan bildu. 
      if (len(i) < baztergarritasuna):
        kendu.extend(i)
        if (len(i) > maxim): #Verbose-rentzat, baztertu den multzo handienaren tamaina eskuratzeko.
          maxim=len(i)

    verbose.append(maxim)

    mantenNodo = len(G.nodes) - len(kendu) #Mantenduko den nodo kopurua kalkulatu. 
    kont=0

    #3. FASEA: Baztertzea erabaki den multzoak baztertu.
    for i in kendu:
      GO.remove_node(i) #Multzoaren nodoa ezabatu.
      kodeak[i]=mantenNodo+kont  #Kodeak hiztegian zenbaki oso bat esleitu baztertua izango den nodo honi, baina onartuak izan direnen ondoren. Izan ere, nodo honek ez du edukiko posiziorik soluzio-bektoreetan.
      kont+=1

    #4. FASEA: Onartu diren multzoen nodoei soluzio-bektoreko posizio bat esleitu eta posizio horren zenbaki osoa ipini haien grafoko identifikazio bezala. 
    kont=0
    for i in GO.nodes: #Grafo prozesatuko nodo bakoitzarentzat
      if (i not in kodeak): 
        ertzak=GO.edges(i) #Eskuratu haren ertzak
        for j in ertzak:
          pisuakGuztira+=GO[j[0]][j[1]][weight] #Modularitatean erabiliko den parametroa eguneratu, grafo prozesatuko pisuen batura.
        kodeak[i]=kont #Hiztegian nodoaren identifikazioaren berri eman.
        mapping = {i:kont} #Grafoko nodoen izen aldaketa egiteko.
        kont+=1
        GO = nx.relabel_nodes(GO, mapping) #Grafoko nodoen izen aldaketa egiteko.

    return GO, kodeak, pisuakGuztira, verbose

k=10
G, kodeak, pisuakGuztira, verbose = prozesatu_grafoa(GO,k)

### Helburu-funtzioa

In [ ]:
from itertools import product
import community
import networkx as nx
import numpy as np
import random
import time

#Partizio baten kanpo eta barne pisuak kalkulatzen ditu
def parPisuak (G,soluzioa,k,weight = 'weight'):
  kideak = np.where(soluzioa == k)[0] #K partizioko kideak lortu
  bpisuak = 0
  kpisuak = 0
  for i in range(len(kideak)): #Kide bakoitzerako
    ertzak = G.edges(kideak[i]) #Bere mugakideak eskuratu
    for j in ertzak:
      if (j[1] in kideak): #Mugakidea uneko partizioko kidea bada
        bpisuak+=G[kideak[i]][j[1]][weight] #Barne pisuetara
      else: #Ez bada
        kpisuak+=G[kideak[i]][j[1]][weight] #Kanpo pisuetara
  return(bpisuak/2,kpisuak) #Barne pisuak zati bi egin behar dira, nodo batetik bestera iritsi bada, alderantziz ere egin delako. Kanpo pisuen kasuan ez, kanpoko nodotik barrukorako ertza ez delako aztertu.  

def modularitatea(G, soluzioa, pisuakGuztira):
  time_start = time.time()
  partizioKop=max(soluzioa)+1
  modularitateak = [ ]
  #0 partizioan ahalik eta nodo gutxien egotea nahi dugu, zigortu, beraz, partizio honetan sailkatzea.
  for i in range(1,partizioKop):
    bpisuak, kpisuak = parPisuak(G,soluzioa,i)
    modularitateak.append(bpisuak/pisuakGuztira - ((kpisuak + 2*bpisuak)/(2*pisuakGuztira))**2)
  time_end = time.time()
  # print("Modularitatearen denbora: ", time_end - time_start)
  return sum(modularitateak)

### Soluzioen bideragarritasuna aztertzeko funtzioa

In [ ]:
def soluzioa_bideragarria_da (soluzioa,partizio_kopurua,grafoa):
  partizio_kop = max(soluzioa)+1 #Partizio kopurua
  #ZIURTATU: Eskatutako partizio kopurua eta soluzioak dituenak bat datozela. 
  if (partizio_kop != partizio_kopurua):
    #print("SOLBID: Soluzioa ez da bideragarria. Eskatutako partizio kopuruaren desberdina.",partizio_kop,partizio_kopurua)
    return(False)
  partizioen_elem_kop = np.zeros(partizio_kop) #Partizio bakoitzean dagoen autore kopurua
  #ZIURTATU: komunitate bakoitzean gutxienez bi pertsona daude. 
  for j in range(len(soluzioa)): #Soluzio bakoitzerako
    if (soluzioa[j] >= 0): 
      partizioen_elem_kop[soluzioa[j]]+=1 #Batu bat uneko autoreari esleitu zaion partizioan
  if (sum(partizioen_elem_kop < 2) >= 2): #Partizioren batek autore bat edo gutxiago baditu, soluzioa ez da bideragarria. (0 partizioak posible du ez eduki kiderik)
    return(False)
  return(True)

## Kargatu eta aurreprozesatu instantzien fitxategiak

In [ ]:
G1 = nx.read_graphml("./Esperim/grafoa159.graphml") #Grafoen karga
G2 = nx.read_graphml("./Esperim/grafoa212.graphml")
G3 = nx.read_graphml("./Esperim/grafoa251.graphml")
G4 = nx.read_graphml("./Esperim/grafoa263.graphml")
G5 = nx.read_graphml("./Esperim/grafoa265.graphml")

G1, kodeak1, pisuakGuztira1, verbose = prozesatu_grafoa(G1,k) #Grafoen aurreprozesaketa
G2, kodeak2, pisuakGuztira2, verbose = prozesatu_grafoa(G2,k)
G3, kodeak1, pisuakGuztira3, verbose = prozesatu_grafoa(G3,k)
G4, kodeak2, pisuakGuztira4, verbose = prozesatu_grafoa(G4,k)
G5, kodeak1, pisuakGuztira5, verbose = prozesatu_grafoa(G5,k)

## Definitu aztertu nahi den algoritmoa

### Suberaketa simulatua

In [ ]:
import math
import random as rm
import itertools as it
def eguneratuTenperatura (hasierakoT,oraingoT,iterazioa):
  oraingoT = hasierakoT / (iterazioa+1)
  return(oraingoT)

def gelditu (tenperatura,minimoa,ebal,ebal_kop):
  if (tenperatura < minimoa or ebal >= ebal_kop):
    return(1)
  return(0)

def orekaLortuDa (hobekGabe,atal):
  if (hobekGabe > atal):
    return(1)
  return(0)

def soluzioaOnartuBeharDa (hautagaia,hautagaiaF,onartuaF,tenperatura):
  energia_diferentzia = onartuaF - hautagaiaF
  onarpena = math.exp(-energia_diferentzia/ tenperatura)
  #print(hautagaiaF, onartuaF, onarpena)
  return(onarpena)

#SWAP INGURUNEA
#Swap inguruneak soluzioko edozein digitu beste edozeinekin trukatuta sor
#daitezkeen soluzio guztiak barne hartzen ditu.
def swap_ingurunea(sol,G,k):
  soluzioak = []
  for i in range(len(sol)):
    for j in range(i+1,len(sol)): #i-n dagoen elementua j-n jarri eta j-n dagoena, i-n
      if (sol[i] == sol[j]):
        continue
      nes = sol.copy()
      a = nes[i]
      nes[i] = nes[j]
      nes[j] = a
    if (soluzioa_bideragarria_da(sol,k,G)):
      soluzioak.append(nes)
    else:
        print("EZ")
  return soluzioak

#Bilaketa lokalaren inguruneak eratzeko honako funtzioak erabili daitezke:
#HAMMING INGURUNEA
#Hamming inguruneak uneko soluzioari digitu bat aldatuta sor daitezkeen
#soluzio guztiak barne hartzen ditu.
def hamming_ingurunea(sol,G,k):
  solutions = []
  for i in range(len(sol)): #soluzioko elementu bakoitzerako
    s = sol.copy()
    for j in range(1,k+1):
      s[i] = j
      if soluzioa_bideragarria_da(sol,k,G): #soluzioa bideragarria bada, ingurunean sartu
        solutions.append(s)
  return solutions

def  suberaketaSimulatua (hasierakoBeroa,hoztuDa,atalasea,ebal_kop,G,k,pisuakGuztira):
  kont = 0 #Lehen begiztaren iterazio kontagailua
  ebal = 0
  tenperatura = hasierakoBeroa #Tenperatura gordeko duen aldagaia
  if (k > len(G.nodes)//2):
    return(0,None)
  best_solution = list(np.random.permutation(k)) + list(np.random.permutation(k)) +  list(np.random.randint(1,k,len(G.nodes)-(2*k)))
  best_solution = np.array(best_solution)
  #best_solution[k:len(G.nodes)] = np.random.randint(1,k,len(G.nodes)-k) #Soluzio onena ausazko hasierako soluzio bat da
  best_fitness = modularitatea(G,best_solution,pisuakGuztira) #Helburu funtzioaren bidez ausazko soluzio horren egokitasuna neurtuko dugu
  uneko_sol = best_solution #begizta barruan erabiltzeko aldagaiak
  uneko_fitness = best_fitness #uneko iteraziora arte aurkitu den soluzio onena (bigarren begiztan), ez du zertan inoizko onena izan
  aurrekoIngurunea = swap_ingurunea(best_solution,G,k) #Inguruneak birziklatu eta kalkuluak aurrezteko, beti erabili berri den ingurunea gorde.
  ingurune_onena = aurrekoIngurunea #Ingurune onena gorde, kalkuluak aurrezteko lehen iteraziotik bigarrenera igarotzean.
  while (gelditu(tenperatura,hoztuDa,ebal,ebal_kop) == 0): #Gelditze irizpidea, tenperatura minimora iritsi gara?
    hobekGabe=0 #Bigarren begiztari hasiera eman ahal izateko, hobekGabe behar bezala hasieratu
    aurrekoIngurunea = ingurune_onena #Aurreko ingurunea hasieratu orain arteko soluzio onenaren gordetako ingurunearekin, bigarren begizta ingurune horretan hasiko baita.
    hautagaia= best_solution
    uneko_sol = best_solution
    while (orekaLortuDa(hobekGabe,atalasea) == 0): #Oreka irizpidea, ez da soluzio hoberik aurkitu atalasea iterazioetan?
      hautagaia = random.choice(aurrekoIngurunea)
      hautagaia_fitness = modularitatea(G,hautagaia,pisuakGuztira) #Uneko inguruneko soluziorik onenaren egokitasuna neurtu
      ebal += 1
      if (ebal == ebal_kop):
        return((best_fitness,best_solution)) 
      #print("Orain arte ebaluazio kopurua:",ebal)
      onartu = soluzioaOnartuBeharDa (hautagaia,hautagaia_fitness,best_fitness,tenperatura) #Soluzioa onargarria den edo ez begiratu
      if (onartu > 1): #Hala bada eta gainera, inoizko onena bada
        best_solution=hautagaia #Soluzio onena bezala hartu
        best_fitness=hautagaia_fitness
        uneko_sol = hautagaia
        uneko_fitness = hautagaia_fitness
        ingurune_onena = swap_ingurunea(best_solution,G,k) #Ingurune berria kalkulatu, zeina ingurune onena ere izango den
        aurreko_ingurunea = ingurune_onena  
        hobekGabe=0 #Hobekuntza bat gertatu da, berrabiarazi oreka-kontagailua, 0 iterazioetan jarraian aurkitu dira unekoa baino soluzio hobeak
        #print("Hobekuntza:",best_fitness)
      elif (rm.random() < onartu): #Soluzioa ez bada inoizko onena, baina onartzea erabaki bada
        uneko_sol = hautagaia
        uneko_fitness = hautagaia_fitness
        hobekGabe+=1 #Ez da hobekuntzarik lortu iterazio honetan, beraz, iterazio bat gehiago jarraian hobekuntzarik lortu gabe
        aurrekoIngurunea = swap_ingurunea(best_solution,G,k) #Hurrengo ingurunea kalkuatu, onartutako soluzioarena, hain zuzen 
        #print("Okertu: ",uneko_fitness, " (",best_fitness,") ")
      else: #Soluzioa ez bada onartu
        hobekGabe+=1 #Hobekuntzarik gabeko beste iterazio bat
        #print("Ukatu: ",hautagaia_fitness, " (",best_fitness,") ")
      #print(hautagaia)  
    kont+=1 #Tenperaturaren eguneraketarako, iterazio bat gehiago igaro da.
    tenperatura= eguneratuTenperatura(hasierakoBeroa,tenperatura,kont) #Tenperatura eguneratu, oreka lortu baita.
    #print("\nOREKA AURKITU DA, tenperatura berria: ",tenperatura,". ",kont," iterazioa")
  return((best_fitness,best_solution))


### UMDA

In [ ]:
# Funtzio honek banaketa modelo bat lagintzen du soluzio berri bat sortzeko
# model parametroak soluzioa sortzeko laginduko den modeloa adierazten du (kasu honetan matrize bat)
# size parametroak soluzioaren tamaina adierazten du
def createSolution(model,size):
  sol_berria = [-1]*size # soluzioaren lista hasieratu
  for a,i in enumerate(model):
    p = np.random.uniform(0,1) # ausaz 0 eta 1 tarteko zenbaki bat sortu
    geh = 0 # gehiketa egiten joateko aldagaia
    for ind,j in enumerate(i):
      geh += j # geh aldagaiari orain arteko baloreak gehitu
      if p <= geh: # orain arteko baloreak ausazko zenbakia baino handiagoa edo berdina bada
        sol_berria[a] = ind + 1 # soluzioko a. elementuan tokatu zaion balioa jarri
        break
  return sol_berria # soluzio berria bueltatu

# Funtzio honek bi lista hartzen ditu parametro bezala eta soluzioak lista baloreak listaren arabera ordenatzen du
# soluzioak parametroak ordenatu nahi dugun lista adierazten du
# baloreak parametroak "soluzioak" lista ordenatzeko kontuan eduki behar diren balioak dituen lista adierazten du
def sortByModularity(baloreak,soluzioak):
  return [(sol,mod) for mod, sol in sorted(zip(baloreak, soluzioak))] #soluzioak lista baloreak listaren arabera ordenatu (soluzioak modularitateen arabera ordenatu)

# G parametroa problemaren grafoa da
# pisuakGuztira parametroa grafoko ertz guztien pisuen balioen gehiketa adierazten du
# k parametroa soluzioak zenbat komunitatetan banatuta egotea nahi dugun adierazten du
# max_iter parametroa algoritmoak egin behar dituen iterazioen kopurua adierazten du
# pop_size parametroa populazioaren tamaina adierazten du
def EDA(G,pisuakGuztira,k,max_iter=1000, pop_size=100):
  k = k-1 #komunitate bat gutxi kolaboratu duten autoreentzat erreserbatu dugu, beraz, bakarrik k-1 komunitate esleitu beharko ditugu hemen
  iter = 0 #iterazio kopurua hasieratu
  pop_size = int(pop_size) #populazio tamaina hasieratu
  population = [] 
  populazioaren_modularitateak = []
  avg = [] #batazbesteko modularitateen lista hasieratu
  hoberenak = [] #modularitate hobeen lista hasieratu
  prob_model = len(G)*[[1/k]*k] #hasierako probabilitate modeloa uniformea da. k komunitate daudenez, horietako bakoitza izateko 1/k probabilitatea dago
  for i in range(pop_size): #lehenengo populazioa sortu
      sl = createSolution(prob_model,len(G))
      population.append(sl) #modeloa lagindu eta populazioan sartu
      populazioaren_modularitateak.append(modularitatea(G,np.array(sl),pisuakGuztira))
      iter+=1

  while iter < max_iter: #iterazio maximora ailegatu ez den bitartean
#     print(f'{iter}/{max_iter}') #oraingo iterazioa idatzi

    nppop = np.array(population)
    s = len(nppop)
    for i,column in enumerate(nppop.T): #aldi berean populazioko soluzio guztiak elementuz-elementu zeharkatu
      vec = [0]*k
      for element in column: #komunitate bakoitza zenbat aldiz agertzen den kontatu
        vec[element-1] += 1/s #bektore honetan zenbatuko da komunitate bakoitza i. nodoari zenbat aldiz esleitu zaion populazioko soluzioetan
      prob_model[i] = vec #kalkulatutako modelo berria normalizatu eta probabilitate modeloa eguneratu

    soluzio_berriak = []
    berrien_modularitateak = []
    for i in range(int(pop_size/2)): #pop_size/2 soluzio berri sortu
      sol = createSolution(prob_model,len(G)) #soluzio bat sortu
      soluzio_berriak.append(sol) #soluzio berrietan gorde
      berrien_modularitateak.append(modularitatea(G,np.array(sol),pisuakGuztira))
      iter+=1
      if iter > max_iter: #iterazio kopurua pasatu bada, amaitu
        break
    if iter > max_iter: #iterazio kopurua pasatu bada, amaitu
      break

    extended_population = population.copy()
    extended_population.extend(soluzio_berriak) #soluzio berriak eta populazioa konbinatu
    
    pop_berrirako = populazioaren_modularitateak.copy()
    pop_berrirako.extend(berrien_modularitateak) #populazioaren modularitateak eta soluzio berrien modularitateak konbinatu

    population, populazioaren_modularitateak = zip(*sortByModularity(pop_berrirako,extended_population)[int(pop_size/2):]) #soluzio berrien eta populazioaren konbinaketaren arteko pop_size soluzio hoberenak populaziora pasatu 
    population = list(population) #populazioa tuplatik listara pasatu
    populazioaren_modularitateak = list(populazioaren_modularitateak) #populazioaren modularitateak tuplatik listara pasatu
    
    ## Beheko kodea ez da beharrezkoa algoritmoaren ibilbidea irudikatu nahi ez bada
#     hoberena = []
#     fit_hoberena = float("-inf")
#     av = 0
#     for i in population: #iterazio bakoitzean soluzio hoberenaren modularitatea eta batazbesteko modularitatea kalkulatu grafikoa egiteko
#       m = modularitatea(G,np.array(i),pisuakGuztira)
#       av += m
#       if m > fit_hoberena: #oraingo soluzioa orain arteko maximoa baino moduluaritate handiagoa badu, maximoa oraingoa da
#         fit_hoberena = m
#         hoberena = i
#     hoberenak.append(fit_hoberena) #modularitate hobeen lista
#     avg.append(av/len(population)) #populazioko batazbesteko modularitatearen lista
  hoberena = populazioaren_modularitateak[-1]
  return hoberena#, hoberenak, avg

### Algoritmo eraikitzailea

In [ ]:
def sortu_soluzioa (grafoa,k):
  ausazko_hasieraPuntuak = [ ] #Partizio bakoitzaren nukleoa
  soluzioa = np.full(len(grafoa.nodes),-1) #Hasieran soluzioak -1 balioak bakarrik ditu, autore esleitu gabeak.
  for i in range(1,k): #Partizio kopurua adina aldiz
    while True: #Hasierako nodo egokiak aurkitu arte egin
      puntua = random.randint(0,len(grafoa.nodes)-1) #Hautagai bat sortu
      if (puntua not in ausazko_hasieraPuntuak and len(list(grafoa.edges(puntua))) > 0 and soluzioa[list(grafoa.edges(puntua))[0][1]] == -1 and soluzioa[puntua] == -1): #Hautagaiak ez du jada aukeratuta egon behar. Hautagaiak gutxienez lotura bat eduki behar du (partizio bakoitzak gutzienez 2 kide). Hautagaiak duen loturen artetik guxtienez bat ez da izan behar beste hasiera puntu bat edo haren ondoko erreserbatu bat. Gainera, hasierako puntuak ezin du beste hasierako puntu batek erreserbatuta egon.  
        ausazko_hasieraPuntuak.append(puntua) #Gehitu hasierako puntua
        unekoa = list(grafoa.edges(ausazko_hasieraPuntuak[i-1])) #Eskuratu bere loturak
        soluzioa[unekoa[0][1]]=i #Erreserbatutako hasiera puntuaren ondoko nodoa i partiziokoa egin
        soluzioa[ausazko_hasieraPuntuak[i-1]]=i #Hasiera puntua i partiziokoa egin
        break #Bukatu uneko partizioaren hasierako puntuaren bilaketa
  #HASIERAKO PUNTUEN HEDAPENA
  itxaron_zerrenda = [ ] #Exekuzioa antolatuko duen ilara hasieratu 
  for i in range(1,k): #Hasierako puntu bakoitzarentzat
    itxaron_zerrenda.append((i,grafoa.edges(ausazko_hasieraPuntuak[i-1]))) #Gehitu bere ondoko nodoak itxaron zerrendara
    itxaron_zerrenda.append((i,grafoa.edges(list(grafoa.edges(ausazko_hasieraPuntuak[i-1]))[0][1]))) #gehitu hasierako puntuaren ondoko erreserbatuaren mugakide nodoak ere zerrendara, bide esklusiboak izan ditzakeelako.
  while len(itxaron_zerrenda) > 0: #Itxaron zerrendan zerbait dagoen bitartean
    random.shuffle(itxaron_zerrenda) #Itxaron zerrenda barajatu, partizio bakarra ez dadin hedatu uneoro grafoan barrena.
    unekoarenErtzakElem = itxaron_zerrenda.pop(0) #Ilarako lehenengo elementua hautatu eta kendu
    unekoarenErtzak = list(unekoarenErtzakElem[1]) #Elementu honek 2 zati ditu, iturburuko elementuaren partizioa [0] eta iturburu horretatik zeuden loturak [1]. Loturak eskuratu.
    for i in unekoarenErtzak: #Lotura bakoitzerako egin:
      if (soluzioa[i[1]] == -1): #Loturak adierazten duen nodoak oraindik partiziorik ez badu esleituta:
        soluzioa[i[1]]=unekoarenErtzakElem[0] #Elementu honen partizioa esleitu. 
        itxaron_zerrenda.append((unekoarenErtzakElem[0],grafoa.edges(i[1]))) #Nodo honetara iritsi garen lehen aldia denez, itxaron zerrendara gehitu bere loturak eta iturburuaren partizioa.
  for i in range(len(soluzioa)): #Baliteke grafoak loturarik gabeko nodoak edukitzea
    if soluzioa[i] == -1: #Haietara oraindik ez da inoiz iritsi algoritmoa
      soluzioa[i]=0 #Esleitu 0 izeneko partizioa, partizio nulua 
  return(soluzioa) #Itzuli soluzioa

#Eraikitzaileak ebaluazio kopuru maximoa egin dezan sortu da funtzio hau, ebaluazio kopuru maximoa aldiz deitzen
#du algoritmo eraikitzailea eta horien artean lortu den soluzio onena itzuli.
def eraikitzailea (grafoa,pisuak_guztira,k,iter_kop):
    onena = -1000
    for i in range(iter_kop):
        sol=sortu_soluzioa(grafoa,k)
        fit=modularitatea(grafoa,sol,pisuak_guztira)
        if (fit > onena):
            onena=fit
    return(fit)

## Definitu CSVak sortzeko azpiprograma

In [ ]:
import csv
#Errenkada berri bat gehituko du CSV fitxategian, errenkada eta irteera fitxategiaren bide_izena sartuta. 
def gehitu_errenkada (errenkada,biz): 
    with open(biz, 'a', newline='') as csvfile:
        spamwriter = csv.writer(csvfile)
        spamwriter.writerow(errenkada)

#Iterazio kopurua, lehen k, azken k, k-ren pausoa, ebaluzio kopurua exekuzioko, instantziak, haien pisuak eta bide-
#izena sartuta, exekuzioak egin eta CSV batean gordeko ditu helburu-funtzioaren balioak.
def exekutatzailea (iter_kop,k1,k2,kz,ebal_kop,instantziak,pisuakGuztira,biz):
    for i in range (k1,k2,kz): #K bakoitzeko
        for m in range(len(instantziak)): #Instantzia bakoitzeko
            izena = [str(i) + "-" + str(m)] #Haren errenkada-izena sortu
            instantziaExek = [ ] #Haren exekuzioen lista hasieratu
            for j in tqdm(range(iter_kop)): #Exekuzio bakoitza egin
                ssim=eraikitzailea(instantziak[m],pisuakGuztira[m],i,ebal_kop) #Aldatu, nahi denean, UMDA edo suberaketa simulatuarekin.
                instantziaExek.append(ssim)
            gehitu_errenkada (izena + instantziaExek,biz) #Exekuzioen berri eman irteera fitxategian

## Deitu goiko azpiprogramari nahi den azterketaren inguruko xehetasunekin

In [ ]:
instantziak = [G1,G2,G3,G4,G5] #Exekuzioak egiteko erabiliko diren instantziak
pisuakGuztira = [pisuakGuztira1,pisuakGuztira2,pisuakGuztira3,pisuakGuztira4,pisuakGuztira5] #Haien pisuak

exekuzio_kop = 25 #K bakoitzeko instantzia bakoitzean zenbat exekuzio egin behar diren
lehen_k = 3 #Zenbateko partizio-kopuruarekin (K) hasi nahi diren exekuzioak 
azken_k = 45 #Zenbateko partizio-kopuruarekin (K) amaitu nahi diren (azken hau ez barne)
pausoa = 3 #K bakoitzaren artean zenbateko pausoa edo tartea utzi nahi den
ebal_kop = 5000 #Algoritmoari emango zaion ebaluazio kopuru maximoa
bide_izena = "./" #Irteerako fitxategiaren bide-izena

exekutatzailea(exekuzio_kop,lehen_k,azken_k,pausoa,ebal_kop,instantziak,pisuakGuztira,bide_izena)